In [1]:
# https://www.kaggle.com/arpandhatt/siamese-neural-networks
# https://keras.io/examples/mnist_siamese/

from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, Dense, MaxPooling2D, Dropout, Flatten, Lambda, LeakyReLU, Activation
import keras.backend as K
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [1]:
DATASET_DIR = '../dataset/mnist-siamese-network-pair/'
description = 'siamese-network-mnist'

In [2]:
def base_network(input_shape):
    model = Sequential()
    model.add(Conv2D(5, (3, 3), input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.5))
    model.add(Conv2D(5, (3, 3)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))
    model.add(Conv2D(7, (3, 3)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))
    model.add(Flatten())
    return model

In [4]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

In [5]:
def load_paired_dataset():
    X_train = np.load(DATASET_DIR + 'tr_pairs.npy')
    y_train = np.load(DATASET_DIR + 'tr_y.npy')
    X_test = np.load(DATASET_DIR + 'ts_pairs.npy')
    y_test = np.load(DATASET_DIR + 'ts_y.npy')
    return X_train, y_train, X_test, y_test

In [6]:
X_train, y_train, X_test, y_test = load_paired_dataset()
input_shape = X_train.shape[2:]

In [34]:
input_A = Input(input_shape)
input_B = Input(input_shape)

shared_network = base_network(input_shape)

network_A = shared_network(input_A)
network_B = shared_network(input_B)

# L1_layer = Lambda(lambda tensor:K.abs(tensor[0] - tensor[1]))
function_layer = Lambda(euclidean_distance)

distance_layer = function_layer([network_A, network_B])

x = Dense(16, activation='relu')(distance_layer)
x = Dense(8, activation='relu')(x)
x = Dense(4, activation='relu')(x)
prediction = Dense(1,activation='sigmoid')(x)

model = Model(inputs=[input_A, input_B],outputs=prediction)

In [35]:
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
sequential_7 (Sequential)       (None, 7)            602         input_13[0][0]                   
                                                                 input_14[0][0]                   
__________________________________________________________________________________________________
lambda_7 (Lambda)               (None, 1)            0           sequential_7[1][0]               
          

In [36]:
es = EarlyStopping(monitor='loss',
                  patience=3,
                  verbose=1,
                  mode='min',
                  restore_best_weights=True)
mc = ModelCheckpoint(description + '-best_Weights.h5',
                     monitor='loss',
                    verbose=1,
                    save_best_only=True,
                    save_weights_only=True)

In [1]:
model.fit(x=[X_train[:, 0], X_train[:, 1]], y=y_train, batch_size=128, epochs=10, validation_split=0.1, callbacks=[es, mc])

NameError: name 'model' is not defined

In [38]:
model.evaluate(x=[X_test[:, 0], X_test[:, 1]], y=y_test)

17820/17820 [==============================] - 2s 121us/step


[0.19406876112141058, 0.9234006734274319]